## 에이전트를 MCP 도구로 노출하기

이 과정을 통해서 모델 컨텍스트 프로토콜(MCP)을 통해 에이전트를 도구로 노출하여 MCP 도구를 지원하는 다른 시스템에서 사용할 수 있도록 하는 방법을 보여줍니다.

#### 에이전트를 MCP 서버로 노출

as_mcp_server()를 사용하면 에이전트를 MCP 서버로 노출할 수 있습니다. 그러면, MCP 호환 클라이언트에서 에이전트를 도구로 호출할 수 있습니다.
먼저 MCP 서버로 노출할 에이전트를 생성합니다. 물론, 에이전트에 Function 도구를 추가할 수도 있습니다.

In [ ]:
from typing import Annotated
from agent_framework.openai import OpenAIResponsesClient

# 오늘의 스페셜 메뉴를 반환하는 도구
def get_specials() -> Annotated[str, "Returns the specials from the menu."]:
    return """
        Special Soup: Clam Chowder
        Special Salad: Cobb Salad
        Special Drink: Chai Tea
        """

# 메뉴 아이템의 가격을 반환하는 도구. 9.99 달러로 고정
def get_item_price(
    menu_item: Annotated[str, "The name of the menu item."],
) -> Annotated[str, "Returns the price of the menu item."]:
    return "$9.99"

# RestaurantAgent 에이전트 생성 및 도구 제공
agent = OpenAIResponsesClient().as_agent(
    name="RestaurantAgent",
    description="Answer questions about the menu.",
    tools=[get_specials, get_item_price],
)